In [1]:
import os,glob
folder_path = './Dataset/CORPUS_TXT/'

for filename in glob.glob(os.path.join(folder_path, '*.txt')):
  with open(filename, 'r',encoding="UTF-8") as f:
    file = f.read()
    print (filename)
    print (len(file))
#file = open("./Dataset/dataset.txt","r",encoding="UTF-8")
    lines = []
    for i in file:
      lines.append(i)
    #print(lines)

./Dataset/CORPUS_TXT\0001a.txt
9654
./Dataset/CORPUS_TXT\0001b.txt
8649
./Dataset/CORPUS_TXT\0001c.txt
10090
./Dataset/CORPUS_TXT\0001d.txt
13708
./Dataset/CORPUS_TXT\0001e.txt
14311
./Dataset/CORPUS_TXT\0002a.txt
12957
./Dataset/CORPUS_TXT\0002b.txt
11154
./Dataset/CORPUS_TXT\0003a.txt
9835
./Dataset/CORPUS_TXT\0003b.txt
8400
./Dataset/CORPUS_TXT\0003c.txt
10299
./Dataset/CORPUS_TXT\0003d.txt
18140
./Dataset/CORPUS_TXT\0003e.txt
18549
./Dataset/CORPUS_TXT\0003f.txt
16734
./Dataset/CORPUS_TXT\0003g.txt
15692
./Dataset/CORPUS_TXT\0003h.txt
16569
./Dataset/CORPUS_TXT\0003i.txt
13749
./Dataset/CORPUS_TXT\0003j.txt
15223
./Dataset/CORPUS_TXT\0003k.txt
38001
./Dataset/CORPUS_TXT\0003l.txt
18137
./Dataset/CORPUS_TXT\0003p.txt
9975
./Dataset/CORPUS_TXT\0004a.txt
10061
./Dataset/CORPUS_TXT\0004b.txt
14072
./Dataset/CORPUS_TXT\0004c.txt
21938
./Dataset/CORPUS_TXT\0004d.txt
71038
./Dataset/CORPUS_TXT\0005a.txt
19502
./Dataset/CORPUS_TXT\0005b.txt
17843
./Dataset/CORPUS_TXT\0005c.txt
36497
./Data

In [ ]:
data = ""
for i in lines:
  data = ' '.join(lines)
print(data)

In [ ]:
data = data.replace('\n',' ').replace('\r',' ').replace('\ufeff',' ')
print(data)

In [ ]:
import string
translator = str.maketrans(string.punctuation,' '*len(string.punctuation))
newdata = data.translate(translator)
print(newdata)
print(type(newdata))

In [ ]:
dictionary = []
for i in newdata.split():
  if i not in dictionary:
    dictionary.append(i)
print(dictionary)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
tokenizer = Tokenizer()
tokenizer.fit_on_texts([newdata])
pickle.dump(tokenizer,open('./token/tokenizer1.pkl','wb'))

In [ ]:
sequence_data = tokenizer.texts_to_sequences([newdata])[0]
#print(sequence_data)
vocab_size = len(tokenizer.word_index)+1
sequences = []
for i in range(3,len(sequence_data)):
  words = sequence_data[i-3:i+1]  #create window of 8 later on
  sequences.append(words)
print(sequences)
text = []
print(vocab_size)
# for i in sequences:
#   text.append(tokenizer.sequences_to_texts([i])[0])
# print(text)

In [9]:
import numpy as np
from tensorflow.keras.utils import to_categorical
sequences = np.array(sequences)
x = []
y = []
for i in sequences:
   x.append(i[0:3])
   y.append(i[-1])
x = np.array(x)
y = np.array(y)
print(y)
y = to_categorical(y, num_classes=vocab_size)
print(x)

[ 12  95 920 ...  19 915 135]
[[ 54 142  57]
 [142  57  12]
 [ 57  12  95]
 ...
 [  3 762  32]
 [762  32  19]
 [ 32  19 915]]


In [11]:
from tensorflow import keras
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
model = keras.Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword3.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [14]:
from tensorflow.keras.optimizers import Adam
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))
model.fit(x, y, epochs=50, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])


C:\Users\KIRTAN\anaconda3\envs\flask\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
350/350 [==============================] - 58s 154ms/step - loss: 6.2587

Epoch 00001: loss improved from inf to 6.25868, saving model to nextword3.h5
Epoch 2/50
350/350 [==============================] - 53s 151ms/step - loss: 5.8552

Epoch 00002: loss improved from 6.25868 to 5.85516, saving model to nextword3.h5
Epoch 3/50
350/350 [==============================] - 52s 150ms/step - loss: 5.6074

Epoch 00003: loss improved from 5.85516 to 5.60736, saving model to nextword3.h5
Epoch 4/50
350/350 [==============================] - 52s 149ms/step - loss: 5.3643

Epoch 00004: loss improved from 5.60736 to 5.36430, saving model to nextword3.h5
Epoch 5/50
350/350 [==============================] - 50s 142ms/step - loss: 5.0910

Epoch 00005: loss improved from 5.36430 to 5.09103, saving model to nextword3.h5
Epoch 6/50
350/350 [==============================] - 49s 139ms/step - loss: 4.8485

Epoch 00006: loss improved from 5.09103 to 4.84846, saving model to nextword3.h5
Epoch 7/

In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer

model = load_model('nextword3.h5')
tokenizer = pickle.load(open('./token/tokenizer1.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    sequence = tokenizer.texts_to_sequences([text])[0]
    print(sequence)
    sequence = np.array([sequence])
    preds = model.predict(sequence)
    preds = np.argmax(preds,axis=1)
    #print(preds)
    predicted_word = ""
    
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
    
    print(predicted_word)
    return predicted_word

In [2]:
while(True):

    text = input("Enter your line: ")
    
    if text == "-1":
        print("Ending The Program.....")
        break
    
    else:
            text = text.split(" ")
            text = text[-3:]
            print(text)
            #text = ''.join(text)
            Predict_Next_Words(model, tokenizer, text)


['i', 'am', '']
[41, 792]
in
['hello', 'how', 'are']
[117, 457]
more
['what', 'are', 'you']
[51, 457, 62]
to
['']
[]


ValueError: in user code:

    C:\Users\KIRTAN\anaconda3\envs\flask\lib\site-packages\tensorflow\python\keras\engine\training.py:1584 predict_function  *
        return step_function(self, iterator)
    C:\Users\KIRTAN\anaconda3\envs\flask\lib\site-packages\tensorflow\python\keras\engine\training.py:1574 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\KIRTAN\anaconda3\envs\flask\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\KIRTAN\anaconda3\envs\flask\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\KIRTAN\anaconda3\envs\flask\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\KIRTAN\anaconda3\envs\flask\lib